# SmartBeds - Filtrados

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [3]:
%matplotlib inline
#para dibujar en el propio notebook
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import pickle as pk #Pikcle as pk
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from utils import dibujado
from sklearn.feature_selection import VarianceThreshold

### Carga de brutos

In [4]:
with open('data/datos_raw.pdd','rb') as f:
    datos_raw = pk.load(f)
datos_seizure = datos_raw.loc[datos_raw['target']==True]

In [5]:
### Días donde hubo crisis
dayOfSeizures = set()

for i in datos_seizure['DateTime']:
    dayOfSeizures.add(i.date())


#Todos los datos de esos días
crit = datos_raw['DateTime'].map(lambda x: x.date() in dayOfSeizures)

datosPart = datos_raw[crit]
datosPart.head()

,DateTime,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,target
3318693,2018-11-10 00:00:00,0.0,24.742268,25.773196,35.051546,28.865979,15.463918,0.0,0.0,0.0,0.0,0.0,0.0,False
3318694,2018-11-10 00:00:00,0.0,29.896907,35.051546,46.391753,37.113402,17.525773,0.0,0.0,0.0,0.0,0.0,0.0,False
3318697,2018-11-10 00:00:01,0.0,31.958763,34.020619,42.268041,30.927835,16.494845,0.0,0.0,0.0,0.0,0.0,0.0,False
3318695,2018-11-10 00:00:01,0.0,31.958763,38.144330,48.453608,39.175258,18.556701,0.0,0.0,0.0,0.0,0.0,0.0,False
3318696,2018-11-10 00:00:01,0.0,34.020619,37.113402,44.329897,34.020619,16.494845,0.0,0.0,0.0,0.0,0.0,0.0,False


In [6]:
## datosPart tiene lo días donde hubo ataques
# Cogemos la hora donde comenzó la crisis y cogemos 30 minutos antes
# Cogemos la hora donde terminó la crisis y cogemos 30 minutos despues
start = [] #Momentos en los que comenzó una crisis
stop = [] #Momentos en los que terminó una crisis

bi = datosPart.first_valid_index()
before = datosPart['target'][bi]
for index,row in datosPart.iterrows():
    if row['target'] and not before:
        start.append(row['DateTime'])
    elif not row['target'] and before:
        stop.append(datosPart['DateTime'][bi])
    before = row['target']
    bi = index
    
halfhour = 20*60

for i in range(len(start)):
    t = start[i]
    s = stop[i]
    start[i] = t-pd.to_timedelta(halfhour,unit='s')
    stop[i] = s+pd.to_timedelta(halfhour,unit='s')

start,stop

([Timestamp('2018-11-10 03:10:00')], [Timestamp('2018-11-10 04:10:00')])

In [7]:
masks = []
for i in range(len(start)):
    t = start[i]
    s = stop[i]
    masks.append(np.array([(datosPart['DateTime'] >= t) & (datosPart['DateTime'] <= s)],dtype=np.bool))

parts = []
mask=sum(masks)[0].astype(np.bool)
datosPartHour = datosPart.iloc[mask]

In [8]:
min_ = np.min(datosPartHour['DateTime'])
max_ = np.max(datosPartHour['DateTime'])
datos_raw = datos_raw[datos_raw['DateTime'].map(lambda x: (x >= min_) & (x <= max_))]

### Funciones

In [11]:
def filter_all(data,window_length, polyorder):
    for i in data:
        data[i] = savgol_filter(data[i],window_length,polyorder)
    return data

### Filtrados

In [12]:
params = [(5,2),(7,2),(9,2),(11,2)]
filts = list()
for p in params:
    dtL = datos_raw.copy()
    dt = filter_all(datos_raw.iloc[:,1:len(datos_raw.columns)-1],p[0],p[1])
    
    sel = VarianceThreshold(threshold=0.5)
    presiones_array = sel.fit_transform(dt)
    presiones_array[presiones_array<1e-4]=0
    
    for i in range(1,len(presiones_array[0])+1):
        dtL['P'+str(i)]=presiones_array[:,i-1]
    listNames = ['DateTime']
    for i in range(1,len(presiones_array[0])+1):
        listNames.append("P"+str(i))
    listNames.append('target')
    dt = dtL[listNames]
    
    filts.append(dt)

P1
P2
P3
P4
P5
P6
P7
P8
P9
P10
P11
P12
P1
P2
P3
P4
P5
P6
P7
P8
P9
P10
P11
P12
P1
P2
P3
P4
P5
P6
P7
P8
P9
P10
P11
P12
P1
P2
P3
P4
P5
P6
P7
P8
P9
P10
P11
P12


In [9]:
for i in range(len(filts)):
    f = filts[i]
    cols = list()
    for j in range(len(f.columns)-2):
        cols.append('P'+str(j+1))
    for c in cols:
        dibujado(datos_raw,f,col=c,title=c)
        plt.savefig('images/'+c+"-savgol("+str(params[i][0])+"-"+str(params[i][1])+").svg")
        plt.close()